#Bird Eye Proyect
Este proyecto utiliza un video de 4 minutos con personas caminando. Utilizando el dataframe recolectado de http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentre-groundtruth.top. Este dataframe contiene la informacion de la ubicacion del bounding box de cada persona por frame.

In [2]:
import pandas as pd
import cv2
import numpy as np
import io
import os
from math import sqrt
import glob
import matplotlib.pyplot as plt
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize

In [3]:
df=pd.read_csv('http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentre-groundtruth.top',names=['numPersona','numFrame','headValid','bodyValid'
                                                                                                                                            ,'headLeft','headTop','headRight','headBottom',
                                                                                                                                            'bodyLeft','bodyTop','bodyRight','bodyBottom']) 

Agregamos una columna Danger para determinar si la persona esta o no en peligro en cierto frame

In [4]:
df['Danger']=False

In [5]:
df.head()

,numPersona,numFrame,headValid,bodyValid,headLeft,headTop,headRight,headBottom,bodyLeft,bodyTop,bodyRight,bodyBottom,Danger
0,0,0,1,1,270.828,794.098,309.037,834.066,235.925,770.142,371.546,1101.029,False
1,1,0,1,1,308.304,302.982,330.837,326.823,285.748,291.418,370.556,493.414,False
2,2,0,1,1,307.919,241.003,328.645,262.841,286.849,230.501,365.794,416.410,False
3,3,0,1,1,739.951,231.693,759.968,253.491,719.708,220.830,786.893,408.145,False
4,4,0,1,1,814.375,246.562,835.021,268.946,793.722,235.410,861.430,427.627,False


In [6]:

df.loc[(df["numFrame"] == 0) & (df["numPersona"] == 0), "Danger"]=True

#Funciones utiles (cada una tiene una explicacion)

In [7]:
def get_box_points(numFrame,numPersona):
    """
    Esta funcion sirve para sacar los puntos del bounding box de cada persona del dataframe
    """
  points=[]
  num_pers=list(df.loc[df['numFrame'] == numFrame-1]['numPersona'])
  f=df.loc[df['numPersona'] == numPersona]
  x1=int(f.loc[f['numFrame']==numFrame]['bodyLeft'])
  x2=int(f.loc[f['numFrame']==numFrame]['bodyTop'])
  x3=int(f.loc[f['numFrame']==numFrame]['bodyRight'])
  x4=int(f.loc[f['numFrame']==numFrame]['bodyBottom'])

  for x in [x1,x2,x3,x4]:
    points.append(x)
  return points

In [8]:
def get_w_h_box(persp_p):
    """
    Sirve para hallar el alto y ancho de la region elegida para hacer el cambio de perspectiva
    """
    x=[]
    y=[]
    for i in range(4):
        x.append(persp_p[i][0])
        y.append(persp_p[i][1])
    max_x=max(x)
    max_y=max(y)
    min_x=min(x)
    min_y=min(y)
    w=max_x-min_x
    h=max_y-min_y
    return h,w

In [9]:
def get_new_points(matrix,width,height):
    """
    Esta funcion sirve para hallar los puntos de las fronteras despues de aplicar la transformacion
    
    """
    borders=np.float32([[0,0],[width,0],[width,height],[0,height]])
    borders = borders.reshape((-1,1,2))
    b_p=cv2.perspectiveTransform(borders,matrix)            
    n_w=int(b_p[2][0][0])
    n_h=int(b_p[2][0][1])
    return b_p,n_w,n_h

In [54]:
def validate_distance(new_c,new_ppl,dt_min=1):
    """
    new_c= Ubicacion de la persona despues de aplicar la transformacion
    new_ppl=Lista con la ubicacion de las demas personas despues de aplicar la transformacion
    dt_min=distancia minima entre personas en metros
    
    """
    flag=0
    dt_min_p=dt_min*160
    for p in new_ppl:
        point=p[0]
        if point != new_c:
            delta_x=new_c[0]-point[0]
            delta_y=new_c[1]-point[1]
            dt=sqrt(pow(delta_x,2)+pow(delta_y,2))
            if dt<dt_min_p: 
                flag=1
                
    if flag==1:
        return True
    
    return False

In [55]:
def change_per(frame,width,height,matrix,ppl):
        """
        frame= Imagen sobre la que se desea dibujar los puntos 
        width= Ancho de la imagen
        height= Alto de la imagen
        matrix= matriz de transformacion en perspectiva
        ppl=Lista que contiene el numero de la persona y la esquina inferior del bounding box de cada persona (esquina, numPers)
        
        """

        b_p,n_w,n_h=get_new_points(matrix,width,height)

        T1 = np.array([[1.0, 0.0, 4000-n_w],[0.0, 1.0, 800],[0.0, 0.0, 1.0]])
    
        matrix=T1@matrix

        top_r=int(b_p[1][0][1])+(2000-n_h)
        frame = cv2.warpPerspective(frame, matrix, (4000,2000))
        new_ppl=[]
        for center in ppl:
            tuple_center=center[0]
            new_c=get_center_ppl(tuple_center,matrix)
            
            new_ppl.append([new_c,center[1]])
        ppl_d=[]
        ppl_nd=[]
        for center in new_ppl:
            danger=validate_distance(center[0],new_ppl,2)
            if danger:        
                cv2.circle(frame, center[0], 20, (0,0,255),-1)
                ppl_d.append(center[1])
            else:
                cv2.circle(frame, center[0], 20, (0,255,0),-1)
                ppl_nd.append(center[1])                
        frame_2=frame[:1600,:4000,:]
        width = int(250)
        height = int(500)
        frame_2 = cv2.resize(frame_2, (width,height), interpolation = cv2.INTER_AREA)
        return frame_2,ppl_d,ppl_nd

In [56]:
def get_center_ppl(center,matrix):
    """
    Sirve para hallar la transformacion de los puntos que se le pasen
    center=Punto al cual aplicar la transformacion
    matrix=Matriz de transformacion aplicada
    """
    p=np.float32([center])
    p=p.reshape((-1,1,2))
    tp=cv2.perspectiveTransform(p,matrix)
    tp[0][0][0]=int(tp[0][0][0])
    tp[0][0][1]=int(tp[0][0][1])
    
    return(tuple(tp[0][0]))

In [57]:
def get_ubication(numFrame,numPersona):
    """
    Esta  funcion sirve para hallar la ubicacion de la persona  (usando la esquina inferior izquierda)
    """
    points=[]
    num_pers=list(df.loc[df['numFrame'] == numFrame-1]['numPersona'])
    f=df.loc[df['numPersona'] == numPersona]
    x1=int(f.loc[f['numFrame']==numFrame]['bodyLeft'])
    x2=int(f.loc[f['numFrame']==numFrame]['bodyTop'])
    x3=int(f.loc[f['numFrame']==numFrame]['bodyRight'])
    x4=int(f.loc[f['numFrame']==numFrame]['bodyBottom'])
    center=[(x1,x4),numPersona]
    return center

In [58]:
import time

#Presentacion y descarga de las imagenes del video (se debe agregar una carpeta "frame" para guardar las imagenes)

Para este proyecto se hizo una transformacion en perspectiva, la perspectiva utlizada fue un cuadrilatero armado con las 2 rectas de la pista, de forma que al hacer la transformacion, estas se vean rectas.

Sin embargo, al hacer la transformacion, se mapea la perspectiva desde el punto (0,0). Para evitar perder informacion se agrega a la matriz de transformacion otra matriz de traslacion, con el fin de mover la imagen hasta que se pueda ver la mayor parte de informacion. La cantidad para trasladar se calcula mapeando los bordes de la imagen original despues de aplicar la transformacion en perspectiva. Con esto, se traslada la imagen hasta que se pueda ver la mayor parte de la imagen.

Finalmente, para determinar si una persona esta o no en peligro, se calcula la distancia haciendo la conversion de la pista a pixeles. En la imagen transformada cada metro equivale a 160 pixeles. En este caso se determino 2 metros como una distancia segura, si los puntos estan mas cerca que esa distancia se colocaran en rojo

In [59]:
capture = cv2.VideoCapture('TownCentreXVID.avi')
frame_count=0
if (capture.isOpened()== False): 
  print("Error opening video stream or file")
total_time_s = time.time()
while(capture.isOpened()):
    ret, frame = capture.read()
    if ret:
        #Medimos el tiempo de ejecucion del programa
        start = time.time()
        height=frame.shape[0]
        width=frame.shape[1]
        frame_count += 1
        black = np.zeros((height, width, 3), np.uint8)
        #Solo se tienen 4501 frames en el dataset
        if frame_count < 4501:
          num_pers=list(df.loc[df['numFrame'] == frame_count-1]['numPersona'])
          ppl=[]
          for pers in num_pers:
            points=[]
            center=get_ubication(frame_count-1,pers)
            ppl.append(center)

        #Estos puntos corresponden a la region utilizada para el cambio de perspectiva
        pt_p = np.float32([[1059,235],[1557,273],[1013,863],[257,723]])
        h,w=get_w_h_box(pt_p)
        target = np.float32([[0,0],[w,0],[w,h],[0,h]])            
        matrix = cv2.getPerspectiveTransform(pt_p, target)
        
        #Esta funcion devuelve el bird eye y una lista de personas que estan o no en peligro
        #ppl_d=Lista de personas en peligro
        #ppl_nd=Lista de personas que no estan en peligro
        bird_eye,ppl_d,ppl_nd=change_per(black,int(frame.shape[1]),int(frame.shape[0]),matrix,ppl)

        if frame_count < 4501:
          for pers in num_pers:
            points=[]
            points=get_box_points(frame_count-1,pers)
            """
            Si la persona esta en peligro, entonces se dibuja un cuadro rojo, sino un cuadro verde
            Ademas, se agrega al dataframe anterior una lista con informacion de si la persona esta o no en peligro
            """
            if pers in ppl_nd:
                cv2.rectangle(frame, (points[0], points[3]),(points[2], points[1]), (0,255,0), thickness=3)
                df.loc[(df["numFrame"] == frame_count-1) & (df["numPersona"] == pers), "Danger"]=False
            else:
                cv2.rectangle(frame, (points[0], points[3]),(points[2], points[1]), (0,0,255), thickness=3)
                df.loc[(df["numFrame"] == frame_count-1) & (df["numPersona"] == pers), "Danger"]=True

        frame_2 = cv2.resize(frame, (500,500), interpolation = cv2.INTER_AREA)
        
        horizontal = np.concatenate((frame_2,bird_eye), axis = 1)
        #Debe crear una carperta "frames" para poder guardar las imagenes
        name='frames/image'+str(frame_count-1) + '.jpg'
        cv2.imshow('Frame', horizontal)
        end = time.time()
        cv2.imwrite(name, horizontal)
        if frame_count==4501:
            break
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
total_time_e=time.time()
print('*************************************')
print(total_time_e-total_time_s)
capture.release()
cv2.destroyAllWindows()
#Duracion total del procesamiento 1830 seg (30 mins)
        

*************************************
4.116651773452759


Guardar y descargar el video

In [46]:
#Se ordenan las imagenes en una lista ordenada.
images = list(glob.iglob(os.path.join('frames', '*.*')))

images = sorted(images, key=lambda x: float(os.path.split(x)[1][5:-4]))

In [47]:
#Funcion para generar el video
def make_video(outvid, images=None, fps=30, size=None, is_color=True, format="FMP4"):

    fourcc = VideoWriter_fourcc(*format)
    video = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if video is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            video = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        video.write(img)
    video.release()
    return video

In [48]:
total_time_s=time.time()
#Lugar en el que se quiere guardar el video

outvid = os.path.join("", "out.mp4")
make_video(outvid, images, fps=30)
total_time_e=time.time()


print(total_time_e-total_time_s)

87.85119605064392


In [ ]:
Estadisticas y Graficas utiles

In [60]:
estadistics=[]
for pers in range(231):
    row={}
    J=df.loc[(df["numPersona"] == pers)&(df["Danger"]==True),"numFrame"]
    row['numDangerFrames']=len(J)
    row['Frames_to_sec']=len(J)/30
    if len(J) != 0:
        row['Frame Start']=min(J)
        row['Frame End']=max(J)
    else:
        row['Frame Start']=0
        row['Frame End']=0
    row['numPers']=pers
    estadistics.append(row)
estadistics=pd.DataFrame(estadistics)
#En caso se desee guardar el dataframe en csv
estadistics.to_csv('estadistics.csv')

In [24]:
#El archivo.csv ya esta cargado, en la carpeta del proyecto, en caso no se desee correr el procesamiento de nuevo
estadistics=pd.read_csv('estadistics.csv',index_col=0)

In [61]:
estadistics = estadistics.sort_values('numDangerFrames',ascending=False)

In [62]:
estadistics

,numDangerFrames,Frames_to_sec,Frame Start,Frame End,numPers
190,900,30.000000,3548,4459,190
58,692,23.066667,1085,1776,58
55,689,22.966667,1050,1738,55
196,582,19.400000,3722,4303,196
139,570,19.000000,2395,3459,139
...,...,...,...,...,...
148,0,0.000000,0,0,148
50,0,0.000000,0,0,50
110,0,0.000000,0,0,110
1,0,0.000000,0,0,1


In [26]:
import seaborn as sns

In [27]:
promedio=estadistics['numDangerFrames'].mean()

In [30]:

finaldata={}
finaldata['Persona mas irresponsable']=estadistics.loc[estadistics['numDangerFrames']==max(estadistics['numDangerFrames']),'numPers']
inicio=estadistics.loc[estadistics['numDangerFrames']==max(estadistics['numDangerFrames']),'Frame Start']
finaldata['Frame inicio']=inicio
finaldata['Tiempo en peligro (secs)']=estadistics.loc[estadistics['numDangerFrames']==max(estadistics['numDangerFrames']),'Frames_to_sec']
inicio_s=inicio/30
minuto=inicio_s//60
segundos=inicio_s-(minuto*60)
finaldata['Minuto']=minuto
finaldata['Segundo']=segundos
max_pers_frame=0
max_frame=0
for frame_count in range(4501):
    num_pers=len(list(df.loc[df['numFrame'] == frame_count]['numPersona']))
    if num_pers>max_pers_frame:
        max_pers_frame=num_pers
        max_frame=frame_count
finaldata['Frame con mas personas en peligro']=max_frame
finaldata['Numero de personas en el frame con mas personas']=max_pers_frame
final_df=pd.DataFrame(finaldata)
final_df

,Persona mas irresponsable,Frame inicio,Tiempo en peligro (secs),Minuto,Segundo,Frame con mas personas en peligro,Numero de personas en el frame con mas personas
190,190,3548,30.0,1.0,58.266667,1429,28


In [38]:
#Se hace un plot de las 50 personas con mas numero de frames en peligro
sns.set(font_scale=15)  # crazy big
data_y = estadistics['numDangerFrames'][:50]
data_x=estadistics['numPers'][:50]
fig_dims = (300, 200)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(data_x,data_y,ax=ax)


In [42]:
list_ppl_frame=[]
for frame_count in range(4501):
    list_ppl_frame.append(len(list(df.loc[df['numFrame'] == frame_count]['numPersona'])))

In [41]:
list_frames=list(range(0,4501))

In [44]:
def personasporframes(inicio,final):
    #Esta funcion es para graficar la cantidad de personas que hay en un rango determinado de frames
        #inicio=Frame de inicio
        #final=Frame final
    sns.set(font_scale=15)  # crazy big
    data_y = list_ppl_frame[inicio:final]
    data_x=list_frames[inicio:final]
    fig_dims = (300, 200)
    fig, ax = plt.subplots(figsize=fig_dims)
    sns.barplot(data_x,data_y,ax=ax)


In [45]:
personasporframes(0,100)